In [31]:
from __future__ import print_function
import glob
import numpy as np
import math
from pyhdf.SD import SD, SDC
import matplotlib
#matplotlib.use('TkAgg')
import pylab
import matplotlib.pyplot as plt
import datetime

import subprocess
from numpy import genfromtxt
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly.graph_objs as go
from scipy.spatial import cKDTree
import bs4
import itertools
import bottleneck as bn
from scipy.signal import lfilter
from itertools import compress
from numpy import inf
import pandas as pd

In [84]:
from operator import itemgetter 
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from collections import OrderedDict

In [32]:
#DATA = np.load('/home/kumarv/tayal007/MODIS_TILES/h08v05.npy')
#DATA = np.load('/home/kumarv/tayal007/MODIS_TILES/h08v05.npy')
#load ndvi data from 2013
DATA = np.load('/Users/kshitij/Downloads/data2013.npy')
#load mask from landsat product
mask = np.load('/Users/kshitij/Downloads/mask.npy')

In [4]:
#data2013 = np.copy(DATA[:,:,586:631])
#np.save('/home/kumarv/tayal007/data2013', data2013)

In [33]:
def readModisfile(path):
    global mask
    file = SD(path, SDC.READ)
    sds_obj = file.select('Land_Cover_Type_1') 
    mask = np.array(sds_obj.get())
    return mask
def percentage(part, shape):
    whole = shape[0] * shape[1] * shape[2]
    return 100 * float(part)/float(whole)

def cleanNDVI(NDVISTACK):
	print ("Missing value in raw data is ", np.count_nonzero(np.isnan(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isnan(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Infinity value in raw data is ", np.count_nonzero(np.isinf(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isinf(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Greater than 1 is ", np.count_nonzero(NDVISTACK > 1),'which is ' ,percentage(np.count_nonzero(NDVISTACK > 1),NDVISTACK.shape),'%')
	print ("Less than -1 is ", np.count_nonzero(NDVISTACK < -1),'which is ' ,percentage(np.count_nonzero(NDVISTACK < -1),NDVISTACK.shape),'%')
	
	CLEANED_DATA = np.copy(NDVISTACK)
	CLEANED_DATA[NDVISTACK == inf] = np.nan
	CLEANED_DATA[NDVISTACK == -inf] = np.nan
	CLEANED_DATA[NDVISTACK > 1] = np.nan
	CLEANED_DATA[NDVISTACK < -1] = np.nan

	print ("After Cleaning Missing value is ", np.count_nonzero(np.isnan(CLEANED_DATA)),'which is ' ,percentage(np.count_nonzero(np.isnan(CLEANED_DATA)),NDVISTACK.shape),'%')
	return CLEANED_DATA;
data2013 = DATA
data2013[data2013 < 0] = 0
data2013 = cleanNDVI(data2013)
#data2013 = data2013.reshape(data2013.shape[0] * data2013.shape[1], data2013.shape[2])

/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in less


Missing value in raw data is  16338907 which is  6.3035906635802466 %
Infinity value in raw data is  160282 which is  0.06183719135802469 %


/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in greater
  


Greater than 1 is  1946976 which is  0.7511481481481481 %


/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in less
  from ipykernel import kernelapp as app


Less than -1 is  0 which is  0.0 %


/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in greater
/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in less


After Cleaning Missing value is  18285883 which is  7.054738811728395 %


In [133]:
def initializedata(data2013,mask,train_size):
    
    crop_data = data2013[mask==12]
    idx1 = np.random.randint(crop_data.shape[0], size=train_size)
    crop_data= crop_data[idx1,:]
    row_mean = np.nanmean(crop_data, axis=1)
    inds = np.where(np.isnan(crop_data))
    crop_data[inds] = np.take(row_mean, inds[0])
    print (crop_data.shape)  #crop
    
    equal = int(train_size/5)
    
    non_crop_Forest = data2013[np.logical_or.reduce((mask==1,mask==2,mask==3,mask==4,mask==5))]
    print (non_crop_Forest.shape)
    idx2 = np.random.randint(non_crop_Forest.shape[0], size=equal)
    non_crop_Forest= non_crop_Forest[idx2,:]

    non_crop_Woodlands = data2013[np.logical_or(mask==8,mask==9)]
    print (non_crop_Woodlands.shape)
    idx3 = np.random.randint(non_crop_Woodlands.shape[0], size=equal)
    non_crop_Woodlands= non_crop_Woodlands[idx3,:]

    non_crop_Grass = data2013[mask==10]
    print (non_crop_Grass.shape)
    idx4 = np.random.randint(non_crop_Grass.shape[0], size=equal)
    non_crop_Grass= non_crop_Grass[idx4,:]

    non_crop_Shrubs = data2013[np.logical_or(mask==6,mask==7)]
    print (non_crop_Shrubs.shape)
    idx5 = np.random.randint(non_crop_Shrubs.shape[0], size=equal)
    non_crop_Shrubs= non_crop_Shrubs[idx5,:]

    non_crop_unvegetated = data2013[np.logical_or.reduce((mask==13,mask==16,mask==17,mask==15))]
    print (non_crop_unvegetated.shape)
    idx6 = np.random.randint(non_crop_unvegetated.shape[0], size=equal)
    non_crop_unvegetated= non_crop_unvegetated[idx6,:]
    
    non_crop_data = np.r_[non_crop_Forest,non_crop_Woodlands,non_crop_Grass,non_crop_Shrubs,non_crop_unvegetated]
    print (non_crop_data.shape)  #non_crop
    
    
    row_mean = np.nanmean(non_crop_data, axis=1)
    inds = np.where(np.isnan(non_crop_data))
    non_crop_data[inds] = np.take(row_mean, inds[0])
    
    non_crop_data = non_crop_data[~np.isnan(non_crop_data).any(axis=1)]
    crop_data = crop_data[~np.isnan(crop_data).any(axis=1)]
    print (crop_data.shape)
    print (non_crop_data.shape)
    
    FF = np.array(("Forest," * equal).split(',')[:-1])
    WW = np.array(("Woodlands," * equal).split(',')[:-1])
    GG = np.array(("Grass," * equal).split(',')[:-1])
    SS = np.array(("Shrubs," * equal).split(',')[:-1])
    UU = np.array(("unvegetated," * equal).split(',')[:-1])
    non_crop_labels = np.r_[FF,WW,GG,SS,UU]
    
    print (non_crop_labels.shape)  #non_crop labels
    
    crop_y = np.ones((train_size,1))  # 1 for crop classification
    non_crop_y = np.zeros((equal * 5,1)) # 0 for non-crop classification
    
    X = np.r_[crop_data,non_crop_data]
    Y = np.r_[crop_y,non_crop_y]
   
    trainindex = np.r_[idx1,idx2,idx3,idx4,idx5,idx6]
    print ("Done")
    #return crop_data, non_crop_data, non_crop_labels
    return X,Y,non_crop_labels, trainindex

In [153]:
def RandomForest(X,Y,train_size,labels,X1,Y1,labels1,crop_shape):
    
    sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0) #Stratified Sampling

    for train_index, test_index in sss.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
    clf = RandomForestClassifier(n_estimators=100,n_jobs=25,verbose=1 ,max_depth=20, random_state=0)
    clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    
    ti = test_index[test_index > train_size] - train_size
    ls = y_pred[test_index > train_size]
    pos = np.array(np.where(ls==1))
    
    misclassifiedlabel = list(itemgetter(*pos)(labels[ti]))
    print (len(misclassifiedlabel))
    c = Counter(misclassifiedlabel)
    for key, value in sorted(c.items(), key = itemgetter(1), reverse = True):
        print(key, value)


    
    print(confusion_matrix(y_test.ravel(), y_pred))  
    print(classification_report(y_test.ravel(), y_pred))
    
    
    
    y_pred = clf.predict(X1)
    
    ls = y_pred[crop_shape:]
    pos = np.array(np.where(ls==1))
    
    misclassifiedlabel = list(itemgetter(*pos)(labels1))
    print (len(misclassifiedlabel))
    c = Counter(misclassifiedlabel)
    for key, value in sorted(c.items(), key = itemgetter(1), reverse = True):
        print(key, value)
    
    print(confusion_matrix(Y1.ravel(), y_pred))  
    print(classification_report(Y1.ravel(), y_pred))
    
    return clf

In [134]:
X,Y, non_crop_labels,train_index =  initializedata(data2013,mask,50000)

(50000, 45)
(243821, 45)
(372468, 45)
(532066, 45)
(2616793, 45)
(505247, 45)
(50000, 45)
(50000, 45)
(50000, 45)
(50000,)
Done


In [ ]:
RandomForest(X,Y,50000,non_crop_labels,X1,Y1,labels1,184469)

In [139]:
def initializecompletedata(data2013,mask,train_index,train_size):
    
    crop_data = data2013[mask==12]
    idx1 = train_index[0:train_size]
    crop_data = np.delete(crop_data, idx1,axis=0)
    
    row_mean = np.nanmean(crop_data, axis=1)
    inds = np.where(np.isnan(crop_data))
    crop_data[inds] = np.take(row_mean, inds[0])
    print (crop_data.shape)  #crop
    
    equal = int(train_size/5)
    
    non_crop_Forest = data2013[np.logical_or.reduce((mask==1,mask==2,mask==3,mask==4,mask==5))]
    print (non_crop_Forest.shape)
    idx2 = train_index[train_size:train_size+equal]
    non_crop_Forest= np.delete(non_crop_Forest, idx2,axis=0)

    non_crop_Woodlands = data2013[np.logical_or(mask==8,mask==9)]
    print (non_crop_Woodlands.shape)
    idx3 = train_index[train_size+equal:train_size+(2* equal)]
    non_crop_Woodlands= np.delete( non_crop_Woodlands, idx3,axis=0)

    non_crop_Grass = data2013[mask==10]
    print (non_crop_Grass.shape)
    idx4 = train_index[train_size+(2*equal):train_size+(3* equal)]
    non_crop_Grass= np.delete( non_crop_Grass, idx4,axis=0)

    non_crop_Shrubs = data2013[np.logical_or(mask==6,mask==7)]
    print (non_crop_Shrubs.shape)
    idx5 = train_index[train_size+(3*equal):train_size+(4* equal)]
    non_crop_Shrubs= np.delete(non_crop_Shrubs, idx5,axis=0)

    non_crop_unvegetated = data2013[np.logical_or.reduce((mask==13,mask==16,mask==17,mask==15))]
    print (non_crop_unvegetated.shape)
    idx6 = train_index[train_size+(4*equal):train_size+(5* equal)]
    non_crop_unvegetated= np.delete(non_crop_unvegetated, idx6,axis=0)
    
    non_crop_data = np.r_[non_crop_Forest,non_crop_Woodlands,non_crop_Grass,non_crop_Shrubs,non_crop_unvegetated]
    print (non_crop_data.shape)  #non_crop
    
    
    row_mean = np.nanmean(non_crop_data, axis=1)
    inds = np.where(np.isnan(non_crop_data))
    non_crop_data[inds] = np.take(row_mean, inds[0])
    
    non_crop_data = non_crop_data[~np.isnan(non_crop_data).any(axis=1)]
    crop_data = crop_data[~np.isnan(crop_data).any(axis=1)]
    print ("Crop data shape: ", crop_data.shape)
    print ("Non_Crop data shape ", non_crop_data.shape)
    
    FF = np.array(("Forest," * non_crop_Forest.shape[0]).split(',')[:-1])
    WW = np.array(("Woodlands," * non_crop_Woodlands.shape[0]).split(',')[:-1])
    GG = np.array(("Grass," * non_crop_Grass.shape[0]).split(',')[:-1])
    SS = np.array(("Shrubs," * non_crop_Shrubs.shape[0]).split(',')[:-1])
    UU = np.array(("unvegetated," * non_crop_unvegetated.shape[0]).split(',')[:-1])
    non_crop_labels = np.r_[FF,WW,GG,SS,UU]
    
    print (non_crop_labels.shape)  #non_crop labels
    
    crop_y = np.ones((crop_data.shape[0],1))  # 1 for crop classification
    non_crop_y = np.zeros((non_crop_data.shape[0],1)) # 0 for non-crop classification
    
    X = np.r_[crop_data,non_crop_data]
    Y = np.r_[crop_y,non_crop_y]

    print ("Done")
    #return crop_data, non_crop_data, non_crop_labels
    return X,Y,non_crop_labels

NoneType

In [154]:
X1,Y1, labels1 = initializecompletedata(data2013,mask,train_index,50000)

(184469, 45)
(243821, 45)
(372468, 45)
(532066, 45)
(2616793, 45)
(505247, 45)
(4220949, 45)
Crop data shape:  (184469, 45)
Non_Crop data shape  (4220949, 45)
(4220949,)
Done


In [151]:
RandomForest_Complete(X1,Y1,labels1,184469)

AttributeError: 'NoneType' object has no attribute 'predict'

In [150]:
def RandomForest_Complete(X,Y,labels,crop_shape, clf):
    
    y_pred = clf.predict(X)
    
    ls = y_pred[crop_shape:]
    pos = np.array(np.where(ls==1))
    
    misclassifiedlabel = list(itemgetter(*pos)(labels))
    print (len(misclassifiedlabel))
    c = Counter(misclassifiedlabel)
    for key, value in sorted(c.items(), key = itemgetter(1), reverse = True):
        print(key, value)
    
    print(confusion_matrix(y_test.ravel(), y_pred))  
    print(classification_report(y_test.ravel(), y_pred))

In [149]:
np.unique(idx1).shape

(44868,)

In [138]:
crop_data = data2013[mask==12]
print (crop_data.shape)
row_mean = np.nanmean(crop_data, axis=1)
inds = np.where(np.isnan(crop_data))
crop_data[inds] = np.take(row_mean, inds[0])
print (crop_data.shape)

(229337, 45)
(229337, 45)


In [42]:
non_crop_data = data2013[mask!=12]
row_mean = np.nanmean(non_crop_data, axis=1)
inds = np.where(np.isnan(non_crop_data))
non_crop_data[inds] = np.take(row_mean, inds[0])
non_crop_data = non_crop_data[~np.isnan(non_crop_data).any(axis=1)]
crop_data = crop_data[~np.isnan(crop_data).any(axis=1)]
print (crop_data.shape)
print (non_crop_data.shape)

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning:

Mean of empty slice



(229337, 45)
(5530662, 45)


In [43]:
y_pred = clf.predict(np.r_[crop_data,non_crop_data])

[Parallel(n_jobs=25)]: Done 100 out of 100 | elapsed:    9.8s finished


In [44]:
crop_y = np.ones((229337,1))
non_crop_y = np.zeros((5530662,1))
y_test = np.r_[crop_y,non_crop_y]

In [45]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test.ravel(), y_pred))  
print(classification_report(y_test.ravel(), y_pred))  

[[5225028  305634]
 [  18867  210470]]
             precision    recall  f1-score   support

        0.0       1.00      0.94      0.97   5530662
        1.0       0.41      0.92      0.56    229337

avg / total       0.97      0.94      0.95   5759999



In [ ]:
# np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/CROP_8_Auto",crop)
# np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/NON_CROP_8_Auto",noncrop)
# np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/ENC_CROP_8_Auto",encoded_crop)
# np.save("/home/kumarv/tayal007/Encodings/Autoencoder/8/ENC_NON_CROP_8_Auto",encoded_noncrop)
crop_data_LSTM = np.load("/home/kumarv/tayal007/Encodings/LSTM/8/CROP_8_Auto.npy")
non_crop_data_LSTM = np.load("/home/kumarv/tayal007/Encodings/LSTM/8/NON_CROP_8_Auto.npy")

In [ ]:
crop_data_LSTM = crop_data_LSTM.reshape(crop_data_LSTM.shape[0],crop_data_LSTM.shape[1])
non_crop_data_LSTM = non_crop_data_LSTM.reshape(non_crop_data_LSTM.shape[0],non_crop_data_LSTM.shape[1])
print (crop_data_LSTM.shape)
print (non_crop_data_LSTM.shape)


In [ ]:
crop_data_LSTM= crop_data_LSTM[idx1,:]
print (crop_data_LSTM.shape)

In [ ]:
non_crop_data_LSTM= np.r_[non_crop_data_LSTM[idx2,:],non_crop_data_LSTM[idx3,:],non_crop_data_LSTM[idx4,:],non_crop_data_LSTM[idx5,:],non_crop_data_LSTM[idx6,:]]
print (non_crop_data_LSTM.shape)

In [ ]:
X = np.r_[crop_data_LSTM,non_crop_data_LSTM]
crop_y = np.ones((10000,1))
non_crop_y = np.zeros((10000,1))
Y = np.r_[crop_y,non_crop_y]
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
for train_index, test_index in sss.split(X, Y):
       #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100,n_jobs=25,verbose=1 ,max_depth=20, random_state=0)
clf.fit(X_train, y_train.ravel())

In [ ]:
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test.ravel(), y_pred))  
print(classification_report(y_test.ravel(), y_pred))

In [ ]:
y_pred = clf.predict(np.r_[crop_data_LSTM,non_crop_data_LSTM])
#crop_y = np.ones((229337,1))
#non_crop_y = np.zeros((5530663,1))
#y_test = np.r_[crop_y,non_crop_y]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test.ravel(), y_pred))  
print(classification_report(y_test.ravel(), y_pred))

In [16]:
p = ['1','2','3','4','5','6','7','8']
q = np.array([1,3,5])

In [26]:
from operator import itemgetter 
a = [-2, 1, 5, 3, 8, 5, 6]
b = [1, 2, 5]
p = list(itemgetter(*b)(a))


In [59]:
np.array(np.where(a==5)) - 50000

array([[-49998, -49995]])

In [97]:
a = [4, 3, 5, 7, 6, 8]
indices = [0, 1, 4]
np.take(a, indices)

array([-5, -4, -7])

In [125]:
crop_data = data2013[mask==12]
idx1 = np.random.randint(crop_data.shape[0], size=5)
print (idx1)
crop_data= crop_data[~idx1,:]

[ 74930  98753 211054  49685 184637]


In [129]:
row_mean = np.nanmean(crop_data, axis=0)
row_mean

array([0.42756397, 0.4408012 , 0.39167553, 0.46620613, 0.45647126,
       0.42780083, 0.48348436, 0.485371  , 0.48655877, 0.56299984,
       0.5975281 , 0.49826446, 0.58125275, 0.561036  , 0.53447735,
       0.4698804 , 0.4753158 , 0.483499  , 0.46181035, 0.4796102 ,
       0.49807018, 0.47870445, 0.53962374, 0.54279464, 0.51410043,
       0.5470287 , 0.53567713, 0.49545693, 0.49977428, 0.49729759,
       0.4691143 , 0.46372455, 0.4255734 , 0.41084254, 0.36409417,
       0.38607168, 0.33794492, 0.36236554, 0.33191657, 0.3398587 ,
       0.36818433, 0.35249475, 0.3577165 , 0.3037686 , 0.29626158],
      dtype=float32)

In [123]:
b = np.delete(crop_data, idx1,axis=1)

/Users/kshitij/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  """Entry point for launching an IPython kernel.


In [124]:
b.shape

(229337, 45)